<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/6/6f/Dauphine_logo_2019_-_Bleu.png" style="width: 600px;"/> 
</center>  

<div align="center"><span style="font-family:Arial Black;font-size:33px;color:darkblue"> Master Economie Finance </span></div>

<div align="center"><span style="font-family:Arial Black;font-size:27px;color:darkblue">Application Lab – Portfolio Management</span></div>

<div align="center"><span style="font-family:Arial Black;font-size:20px;color:darkblue">Time series data modelling
</span></div>

In [4]:
!pip install pandas > /dev/null 2>&1
!pip install numpy > /dev/null 2>&1
!pip install yfinance > /dev/null 2>&1
!pip install matplotlib > /dev/null 2>&1
!pip install seaborn > /dev/null 2>&1
!pip install arch > /dev/null 2>&1

In [8]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-success">
    <b>EXERCISE: Data download and transformation</b>:
    <ul>
        <li>Download the Close prices for the following tickers: 
            <code>'JPM', 'BAC', 'C', 'GS', 'MS'</code> 
            using <code>yfinance</code>. Set the start date to <code>'2010-01-01'</code> and the end date to <code>'2025-08-31'</code>.</li>
        <li>Transform the Adjusted Close prices into log returns for each stock and times 100.
        <li>Create a new DataFrame named <code>log_returns</code> to store the calculated log returns for all stocks.</li>
        <li>Drop missing values (e.g., NaN values) resulting from the transformation.</li>
        <li>Visualize the Price and log returns of <code>'JPM'</code> using line plot.</li>
    </ul>
</div>

Why is stationary matter?

Check Stationarity (ADF Test)

In [ ]:
from statsmodels.tsa.stattools import adfuller

jpm_price = data['JPM'].to_numpy()

adf_result = adfuller(jpm_price)
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
print("Critical Values:", adf_result[4])

In [ ]:
#We can see that the close price of JPM is not stationary, now test for log returns of JPM
jpm_return = log_returns['JPM'].to_numpy()

adf_result = adfuller(jpm_return)
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
print("Critical Values:", adf_result[4])

## Autoregressive (AR) Model

$$
X_t = \phi_0 + \phi_1 X_{t-1} + \phi_2 X_{t-2} + \dots + \phi_p X_{t-p} + \epsilon_t
$$

### PACF

When modeling real data with an AR model, the order $p$ is unknown. Determining $p$ is called the problem of order selection. Common methods include the partial autocorrelation function (PACF) and the AIC or BIC criterion.

Let $X_1, \ldots, X_n, Y$ be random variables, and define

$$
L(Y|X_1, \ldots, X_n) = \arg\min_{\hat{Y}=b_0+b_1X_1+\cdots+b_nX_n} E(Y-\hat{Y})^2,
$$

which is called the best linear prediction of $Y$ using $X_1, \ldots, X_n$. The correlation coefficient between $Y - L(Y|X_1, \ldots, X_n)$ and $Z - L(Z|X_1, \ldots, X_n)$ is called the **partial correlation coefficient** between $Y$ and $Z$ after removing the effect of $X_1, \ldots, X_n$.

For a stationary linear time series, for $n=1,2,\ldots$, we have

$$
L(X_t | X_{t-1}, \ldots, X_{t-n}) = \phi_{n0} + \phi_{n1} X_{t-1} + \cdots + \phi_{nn} X_{t-n},
$$

where $\phi_{nj}, j=0,1,\ldots,n$ are independent of $t$. $\phi_{nn}$ is the **partial autocorrelation coefficient** of the time series $\{X_t\}$, and the sequence $\{\phi_{nn}\}$ is called the **partial autocorrelation function (PACF)** of the time series $\{X_t\}$.

In fact, $\phi_{nn}$ is the partial correlation coefficient between $X_t$ and $X_{t-n}$ after removing the effect of $X_{t-1}, \ldots, X_{t-(n-1)}$. In particular, $\phi_{11} = \rho_1$.

The sample estimate of $\phi_{nn}$, denoted $\hat{\phi}_{nn}, n=1,2,\ldots$, is called the **sample partial autocorrelation function**. In Python, it can be estimated and plotted using plot_pacf() from statsmodels.

If ${X_t}$ follows the AR($p$) model:

$$
X_t = \phi_0 + \phi_1 X_{t-1} + \cdots + \phi_p X_{t-p} + \varepsilon_t, \quad \phi_p \neq 0,
$$

this means that when predicting $X_t$ using a linear combination of $X_{t-1}, \ldots, X_{t-p}$, only these lags are needed. Adding $X_{t-p-1}, X_{t-p-2}, \ldots$ does not improve the prediction. This implies that $\hat{\phi}_{kk}=0, k>p$. This property is called the cut-off property of the PACF for AR models.


In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(jpm_return, lags=40)
plt.title("PACF Plot")
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(jpm_return, lags=40)
plt.title("ACF Plot")
plt.show()

There are generally 2 ways of fitting a AR model, ARIMA is perfered, standard errors are more precise.

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

ar_model = AutoReg(jpm_data, lags=1).fit()
ar_model.summary()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

ar_model = ARIMA(jpm_data, order=(1, 0, 0)).fit()
ar_model.summary()

### Information Criterion

Akaike Information Criterion

It is defined as:

$$
AIC = -\frac{2}{T}\ln(\text{likelihood value}) + \frac{2}{T}(\text{number of parameters}),
$$

where the likelihood value is the likelihood evaluated at the maximum likelihood estimates of the parameters.  
When the model is a higher-order AR(p), i.e., $\{\varepsilon_t\}$ is an independent $N(0,\sigma^2)$ sequence in the AR(p) model, the AIC formula is:

$$
AIC(k) = \ln \hat{\sigma}_k^2 + \frac{2k}{T},
$$

where $k$ is the order of the model, and $\hat{\sigma}_k^2$ is the maximum likelihood estimate of the variance of $\varepsilon_t$ under order $k$.  
$\ln \hat{\sigma}_k^2$ represents the goodness of fit of the model to the data: the smaller the value, the better the fit.  
The term $\frac{2k}{T}$ is a penalty for model complexity: the larger the $k$, the more complex the model, the less stable it is, and the worse its adaptability to future situations.  
Choosing $k$ that minimizes $AIC(k)$ within a certain range achieves a trade-off between goodness of fit and model simplicity.

---

Bayesian Information Criterion

Another commonly used information criterion is the BIC. For a higher-order AR model:

$$
BIC(k) = \ln \hat{\sigma}_k^2 + \frac{k \ln T}{T},
$$

BIC tends to select a lower-order model than AIC.


In [ ]:
print(ar_model.aic)
print(ar_model.bic)

In [ ]:
ar_model.bic

Prediction for next period

In [ ]:
ar_model.forecast(steps = 1)

<div class="alert alert-success">
    <b>EXERCISE: Find the Best Lag Order Using BIC</b>:
    <ul>        
        <li>Fit an AR model for lag orders ranging from 1 to 10. Hint: Use statsmodels ARIMA function.</li> 
        <li>For each lag order, calculate the BIC values of the fitted AR model.</li>
        <li>Create a list to store the  BIC values corresponding to each lag order.</li>
        <li>Find the lag order with the minimum BIC values.</li>
    </ul>
</div>

Check for residuals

In [ ]:
ar_residuals = ar_model.resid

plt.figure(figsize=(12, 6))
plt.plot(ar_residuals, label='Residuals', color='blue')
plt.title("Residuals Over Time")
plt.xlabel("Date")
plt.ylabel("Residuals")
plt.legend()
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Ljung-Box test
ljung_box_result = acorr_ljungbox(ar_residuals, lags=[np.floor(np.sqrt(len(jpm_return)))], return_df=True)
print(ljung_box_result)

## Moving Average (MA) Model

$$
X_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots + \theta_q \epsilon_{t-q}
$$

The MA (Moving Average) model describes the current value (the value of the time series) as being determined by the weighted sum of random error terms.

Select the lag using Autocorrelation Function(ACF) plot

In [ ]:
plot_acf(jpm_return, lags=40)
plt.title("ACF Plot")
plt.show()

In [ ]:
ma_model = ARIMA(jpm_return, order=(0, 0, 2)).fit()
ma_model.summary()

Prediction for next period

In [ ]:
forecast = ma_model.forecast(steps=1)
forecast

In [ ]:
ma_residuals = ma_model.resid

ljung_box_result = acorr_ljungbox(ma_residuals, lags=[np.floor(np.sqrt(len(jpm_return)))], return_df=True)
print(ljung_box_result)

## Autoregressive Moving Average (ARMA) Model

$$
X_t = \mu + \phi_1 X_{t-1} + \phi_2 X_{t-2} + \dots + \phi_p X_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots + \theta_q \epsilon_{t-q}
$$

In [ ]:
arma_model = ARIMA(jpm_return, order=(2, 0, 2)).fit()
arma_model.summary()

In [ ]:
arma_model.forecast(steps = 1)

In [ ]:
arma_residuals = arma_model.resid

plt.figure(figsize=(12, 6))
plt.plot(arma_residuals, label='Residuals', color='blue')
plt.title("Residuals Over Time")
plt.xlabel("Date")
plt.ylabel("Residuals")
plt.legend()
plt.show()

In [ ]:
ljung_box_result = acorr_ljungbox(arma_residuals, lags=[np.floor(np.sqrt(len(jpm_return)))], return_df=True)
print(ljung_box_result)

## Generalized Autoregressive Conditional Heteroskedasticity (GARCH) Model

$$
X_t = \mu + \epsilon_t
$$

$$
\epsilon_t = \sigma_t z_t
$$

$$
\sigma_t^2 = \alpha_0 + \sum_{i=1}^{q} \alpha_i \epsilon_{t-i}^2 + \sum_{j=1}^{p} \beta_j \sigma_{t-j}^2
$$

In [ ]:
from arch import arch_model

We can choose mean = 'Zero', 'Constant', 'AR'

dist = 'Normal', 't', 'skewt', etc.

In [ ]:
garch_model = arch_model(jpm_return, mean='Zero', vol='GARCH', p=1, q=1, dist='Normal').fit()
garch_model.summary()

In [ ]:
# Extract residuals
eps = garch_model.resid               # epsilon_t
sigma = res.conditional_volatility    # sigma_t
z = res.std_resid                    # standardized residuals: z_t = eps_t / sigma_t

In [ ]:
# Residuals over time
plt.figure(figsize=(10, 3))
plt.plot(eps, lw=0.8)
plt.title("Residuals (epsilon_t)")
plt.xlabel("Time")
plt.ylabel("Residuals")
plt.tight_layout()
plt.show()

# Standardized residuals over time
plt.figure(figsize=(10, 3))
plt.plot(z, lw=0.8)
plt.title("Standardized Residuals (z_t)")
plt.xlabel("Time")
plt.ylabel("z_t")
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import norm, probplot, skew, kurtosis

# Histogram of z with standard normal pdf
plt.figure(figsize=(6, 4))
count, bins, _ = plt.hist(z, bins=50, density=True, alpha=0.5, edgecolor="none", label="z_t histogram")
x = np.linspace(z.min(), z.max(), 400)
plt.plot(x, norm.pdf(x), "k--", label="N(0,1) pdf")
plt.title("Standardized Residuals: Histogram vs N(0,1)")
plt.xlabel("z_t")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

# Normal Q-Q plot for z
plt.figure(figsize=(4.5, 4.5))
probplot(z, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot of z_t")
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import t as student_t, probplot

am_t = arch_model(jpm_return, mean='Zero', vol='GARCH', p=1, q=1, dist='t')
res_t = am_t.fit()

#Extract standardized residuals and estimated df
z_t = res_t.std_resid     # standardized residuals: epsilon_t / sigma_t
df_t = float(res_t.params.get("nu"))  # estimated degrees of freedom

#Histogram with Student-t pdf 
plt.figure(figsize=(6, 4))
plt.hist(z_t, bins=50, density=True, alpha=0.5, edgecolor="none", label="z_t histogram")
x = np.linspace(z_t.min(), z_t.max(), 400)
plt.plot(x, student_t.pdf(x, df=df_t), "r--", label=f"t pdf (df={df_t:.1f})")
plt.title("Standardized Residuals: Histogram vs t pdf")
plt.xlabel("z_t")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

# Student-t Q-Q plot for z_t
plt.figure(figsize=(4.5, 4.5))
probplot(z_t, dist=student_t, sparams=(df_t,), plot=plt)
plt.title(f"t Q-Q Plot of z_t (df≈{df_t:.1f})")
plt.tight_layout()
plt.show()

Prediction for next period

In [ ]:
res_t.forecast(horizon=1).variance.values[0,0]

<div class="alert alert-success">
    <b>EXERCISE:</b>
    <ul>
        <li>Write a loop to find the best GARCH model order (<code>p</code>, <code>q</code>) where <code>p</code> ranges from 1 to 5 and <code>q</code> ranges from 1 to 5, using the Bayesian Information Criterion (BIC) for selection.</li>
        <li>Using a rolling window of 252 observations, predict the volatility for the next period for each window.</li>
        <li>Plot the squared predicted volatility against the squared returns for comparison.</li>
    </ul>
</div>

# Linear regression with python

## Data import

We use the Boston real estate price data.

The first 401 observations are used for the estimation of a linear regression and the following observations for the calculation of the forecasts and the measurement of the precision of these forecasts by various criteria.


We will estimate two regressions:
    
     * regression of 'MEDV' (denoted y) on 'RM' denoted (x)
     * regression of 'MEDV' on all the other variables noted X

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

data = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)
# Separate the data into two sub-samples
# sample 1: up to index 400
# sample 2: from index 401
y1 = data.loc[:400,'MEDV'] #data.iloc[:401,-1]
y2 = data.loc[401:,'MEDV'] #data.iloc[401:,-1]
x1 = data.loc[:400,'RM']  
x2 = data.loc[401:,'RM'] 
X1 = data.loc[:400,data.columns!='MEDV'] 
X2 = data.loc[401:,data.columns!='MEDV'] 

In [ ]:
data

## Simple linear regression with scikit-learn

Estimating a simple linear regression with **scikit-learn** involves 5 steps:
    
     1. import necessary packages and object classes
    
     2. Importing data and performing appropriate transformations
    
     3. Creation and Estimation of a Regression Model
    
     4. Checking the estimation results
    
     5. forecast with the estimated model
    
    
   
    https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Importing packages

In [ ]:
from sklearn.linear_model import LinearRegression

sklearn.linear_model.LinearRegression allows to estimate linear or polynomial regressions and to calculate forecasts with the estimated models.

### Collection of data

We transform the series **y** and **x** into numpy.array

In [ ]:
x=np.array(x1)
y=np.array(y1)

x.shape, y.shape

We must transform x into an array of dimension 2: one column and many rows

In [ ]:
x=x[:,np.newaxis]

In [ ]:
x.shape

### Creating a model and estimating with existing data

In [ ]:
model = LinearRegression()

In [ ]:
model

In [ ]:
model.fit(x,y)

Equivalent writing

In [ ]:
model = LinearRegression().fit(x,y)

### Displaying the results of a regression

#### Estimated coefficients

In [ ]:
print(model.intercept_, model.coef_)#note the underscore

#### $R^{2}$

In [ ]:
r_sq =model.score(x,y)
print(f"determination coefficient: {r_sq}")

### Forecast: calculation of adjusted variables

In [ ]:
x_pred =np.array(x2)
x_pred =x_pred[:,np.newaxis]

In [ ]:
y_pred = model.predict(x_pred)

In [ ]:
print(f"predicted variable:\n{y_pred}")

In [ ]:
# alternative method: by calculating y_pred from the linear regression formula

y_pred2 = model.intercept_ + model.coef_*x_pred
print(f"predicted answer from calculation formula:\n{y_pred2}")

### Calculation of forecast accuracy indicators

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
print("Mean squared error: %.2f" %mean_squared_error(y2,y_pred))
print("Mean absolute error: %.2f" %mean_absolute_error(y2,y_pred))
print("Mean absolute percentage error: %.2f" %mean_absolute_percentage_error(y2,y_pred))

### Graphics with Scikit-learn

In [ ]:
# Use plt.scatter() to plot the actual data and plt.plot() to plot the predicted line
plt.scatter(x2,y2,color="black")
plt.plot(x2,y_pred,color="blue",linewidth=2)
plt.show()

# Multiple linear regression with scikit-learn

In [ ]:
X = np.array(X1)
X.shape

In [ ]:
modelall = LinearRegression().fit(X,y)

In [ ]:
modelall.intercept_, modelall.coef_

In [ ]:
r_sq = modelall.score(X,y)
r_sq

In [ ]:
x_pred = np.array(X2)
yall_pred = modelall.predict(x_pred)

In [ ]:
yall_pred

In [ ]:
print("Mean squared error: %.2f" %mean_squared_error(y2,y_pred))
print("Mean absolute error: %.2f" %mean_absolute_error(y2,y_pred))
print("Mean absolute percentage error: %.2f" %mean_absolute_percentage_error(y2,y_pred))

In [ ]:
mean_squared_error(y2,yall_pred)

In [ ]:
mean_absolute_error(y2,yall_pred)

In [ ]:
mean_absolute_percentage_error(y2,yall_pred)

# Linear regressions with statsmodels

### import the packages

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
X1=sm.add_constant(X1) # statsmodels does not add a default constant in linear regressions. It needs to be added.
regall=sm.OLS(y1,X1).fit()
regall.summary()

In [ ]:
# Display of estimated coefficients
regall.params

In [ ]:
# Display of the t-stat of the estimated coefficients
regall.tvalues

In [ ]:
# Display pvalue of the F-stat
regall.f_pvalue

In [ ]:
#correlation matrix
sns.heatmap(data.corr())
plt.show()

In [ ]:
# If we want to check that the correlation is significantly different from 0
from scipy import stats
stats.pearsonr(data['AGE'].to_numpy(),data['DIS'].to_numpy())

In [ ]:
# Display R² and adjusted R²
regall.rsquared, regall.rsquared_adj

In [ ]:
# Display of residuals
regall.resid

In [ ]:
regall.aic

In [ ]:
dir(regall) # list of all objects that can be retrieved from the results

### specification test and residue tests

https://www.statsmodels.org/stable/stats.html

In [ ]:
sm.stats.stattools.durbin_watson(regall.resid)

### Estimating models with a formula
we use the library **statsmodels.formula.api**

smf.ols allows the direct use of formula strings similar to those in R language to define the model, which can include interaction terms, polynomial terms, and more.

In [ ]:
reg1 = smf.ols('MEDV~CRIM+PTRATIO',data=data).fit() # lowercase ols

In [ ]:
reg1.params

In [ ]:
## display results
reg1.summary()

We present the estimation of a regression by OLS. However, you can also do:
    
* GLS: generalized least squares
    
* WLS: weighted least squares

## Programming a linear regression

### Linear regression model
We assume that we have $P$ pairs of variables $(y_{p},x_{p})$ where $y_{p}$ is the explained variable and $x_{p}=(x_{1p} ,...,x_{Np})^{T}$ is the column vector of the $N$ explanatory variables (for the individual $p$).
These variables are linked by the linear relationship:
$$
y_{p}= w_{0} + \sum_{i}^{N} w_{i} x_{ip} + e_{p} = w_{0} + x_{p}^{T}w + e_{ p}
$$
where $w=(w_{1},w_{2},...w_{N})^{T}$ is the (N,1) vector of coefficients.

We note $w^{*} = (w_{0},w_{1},...,w_{N})^{T}$ the vector of dimensions (N+1,1) containing the constant $w_ {0}$ and $w$.


Build a Python function, called **model(x,w)**, having as arguments (inputs) $x$ and $w^{*}$ and returning (output) the adjusted values $N$ : $w_{0 } + x_{p}'w$

In [ ]:
# Note: we use the np.dot() matrix product of numpy


In statistical modeling and practical implementation, e typically represents the error term or random disturbance. When building predictive models, our goal is to estimate model parameters (i.e., coefficients w and the constant term w0 so that we can use these estimates to predict or explain the dependent variable y.

### Objective function

We note $y=(y_{1}, y_{2},...,y_{P})$ the sequence of $1 \times P$ explained variables and $x = (x_{1},..., x_{P})$ the (N,p) matrix of explanatory variables.

We are looking for the parameters $w^{*}$ which minimize the least squares cost function:
     $$
     c(w^{*},x,y) = \frac{1}{P} \sum_{i=1}^{P} (y_{i} - w_{0}-x^{T}_{ i}w)^{2}
     $$
    
Build a python function **least_squares(w)** having as arguments $w^{*}, x,y$ and returning the cost function $c(w^{*},x,y)$


x =
\begin{bmatrix}
x_{11} & x_{12} & \dots & x_{1P} \\
x_{21} & x_{22} & \dots & x_{2P} \\
\vdots & \vdots & \ddots & \vdots \\
x_{N1} & x_{N2} & \dots & x_{NP} \\
\end{bmatrix}

To avoid making the loop, we can use the Numpy function np.sum which calculates the sum of the elements of an array.



# Excercise

###  Stability of $ \beta_i $

We consider the following decomposition of the return of an asset $ i $:

$$
r_{it} = \alpha_i + \beta_i r_{mt} + e_{it}
$$

where $ r_{it} $ is the return on asset $ i $ and $ r_{mt} $ is the return on the stock market index. The term $ \beta_i r_{mt} $ measures the common variations to all assets, the residual $ e_{it} $ represents the specific risk of each asset.

The value of $ \beta_i $ determines the risk of asset $ i $ relative to the market portfolio:

- $ \beta_i = 1 $: asset $ i $ is as risky as the market portfolio.
- $ \beta_i < 1 $: asset $ i $ is less risky than the market portfolio.
- $ \beta_i > 1 $: asset $ i $ is more risky than the market portfolio.


---
1. **Import Excercise.csv**

2. **Select 10 securities at random.** For each of the $ n = 10 $ securities, estimate by the **OLS regression**: $ r_{it} = \alpha_i + \beta_i r_{mt} + e_{it} $ with $ t = 1, \dots, 40 $ first observations. The estimated model is noted

    $$
    r_{it} = \hat{\alpha}_i + \hat{\beta}_i r_{mt} + \hat{e}_{it}, \quad t = 1, \dots, 40
    $$

    We note $ \beta_{i,40} $ as the $ \beta $ of asset $ i $ obtained from the 40 first observations.

3. **Estimate the same regression** for the equally weighted portfolio of $ n = 10 $ securities $ r_{pt} = \frac{1}{n} \sum_{i=1}^{n} r_{it} $

    $$
    r_{pt} = \alpha_p + \beta_p r_{mt} + e_{pt}, \quad t = 1, \dots, 40
    $$

    It can be shown that $ \alpha_p = \frac{1}{n} \sum_{i=1}^{n} \alpha_i $ and $ \beta_p = \frac{1}{n} \sum_{i=1}^{n} \beta_i $.

4. **Re-estimate the previous regressions for each security and the portfolio** by adding the following observations one by one to the first 40 observations. We will obtain for each asset a sequence of $ \beta_{i,40}, \beta_{i,41}, \dots, \beta_{i,T} $ where $ T $ is the end date of the sample. For each sequence of $ \beta_{i,40}, \beta_{i,41}, \dots, \beta_{i,T} $, calculate:

    - the mean,
    - the minimum, the maximum,
    - the standard deviation
      
5. **Plot the evolution of the estimated betas over time**
Using matplotlib, represent on a single graph the evolution of the coefficients $ \beta_{i,t} $ for the 10 selected securities as the estimation window grows from 40 to $T$.
Each line should show how the estimated beta of a given stock evolves as new data are added to the regression. Commetn on the chart.